In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
os.chdir('../../')

In [3]:
from musicautobot.numpy_encode import *
from musicautobot.config import *
from musicautobot.music_transformer import *

ModuleNotFoundError: No module named 'fastai'

## MusicTransformer Training

MusicTransformer takes the basic idea of [Language Models](https://en.wikipedia.org/wiki/Language_model) and applies it to Music.  

Given a sequence of notes, predict the next most likely set of notes.

This model is based off of [transformer-XL](https://arxiv.org/abs/1901.02860) and uses fast.ai's [implementation](https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py) of it.

In [ ]:
# Location of your midi filesfiles
midi_path = Path('/content/drive/My Drive/musicautobot/data/midi/classical_valid')
midi_path.mkdir(parents=True, exist_ok=True)

# Location to save dataset
data_path = Path('/content/drive/My Drive/musicautobot/data/numpy')
data_path.mkdir(parents=True, exist_ok=True)

data_save_name = 'data_save.pkl'

## 1. Gather midi dataset

Make sure all your midi data is in `musicautobot/data/midi` directory

Here's a pretty good dataset with lots of midi data:  
https://www.reddit.com/r/datasets/comments/3akhxy/the_largest_midi_collection_on_the_internet/

Download the folder and unzip it to `data/midi`

## 2. Create dataset from MIDI files

In [ ]:
midi_files = get_files(midi_path, '.mid', recurse=True); len(midi_files)

In [ ]:
processors = [Midi2ItemProcessor()]
data = MusicDataBunch.from_files(midi_files, data_path, processors=processors, bs=2, bptt=12)
data.save(data_save_name)

In [ ]:
# Show Data
data.train_dl.on_epoch_begin()
x, y = data.one_batch();
x, y

## 3. Load Model

In [ ]:
batch_size = 1
data = load_data(data_path, data_save_name, bs=batch_size)

In [ ]:
config = default_config()
learn = music_model_learner(data, config=config.copy())

## 4. Train

In [ ]:
learn.fit_one_cycle(10)

In [ ]:
learn.save('example')

## 5. Predict

---
See [Generate.ipynb](Generate.ipynb) to use a pretrained model and generate better predictions

---

In [ ]:
midi_file = Path('/content/drive/My Drive/musicautobot/data/midi/notebook_examples/single_bar_example.mid'); midi_file
item = MusicItem.from_file(midi_file, data.vocab);

In [ ]:
item.show()

Here's what the seed sounds like:

### Start Predictions:

In [ ]:
pred = learn.predict(item, n_words=100)

In [ ]:
type(pred[0])

Prediction

In [ ]:
# Prediction
pred[0].show()

In [ ]:
pred[1].show()

In [ ]:
from music21 import *
from datetime import *

In [ ]:
len(pred)

In [ ]:
i = 0
while i < len(pred):
  s = pred[i].to_stream(120)
  s.write('midi', '/content/drive/My Drive/musicautobot/generated_music/'+datetime.now().strftime("%d%m%H%M%S")+'.mid')
  i += 1